https://www.annytab.com/extract-text-from-pdf-or-image-in-python/

In [10]:
!pip install pytesseract
!pip install image
!pip install pdf2image

In [11]:
import pandas as pd
import numpy as np
import pytesseract
import image
import os

In [27]:
os.environ["TESSDATA_PREFIX"] = "/home/andrewcgaitskell/Documents/Code/document-image-scanning"

In [28]:
tessdata_dir_config = '--tessdata-dir "/home/andrewcgaitskell/Documents/Code/document-image-scanning"'
##img_path='screenshot.png'
##text=pytesseract.image_to_string(Image.open(img_path), config=tessdata_dir_config)

In [29]:
# Import libraries
import pandas as pd
import pytesseract as pt
import pdf2image
# Read a pdf file as image pages
# We do not want images to be to big, dpi=200
# All our images should have the same size (depends on dpi), width=1654 and height=2340
pages = pdf2image.convert_from_path(pdf_path='data.pdf', dpi=200, size=(1654,2340))
# Save all pages as images
for i in range(len(pages)):
    pages[i].save('image_' + str(i) + '.jpg')
# Convert a page to a string (page 2)


In [30]:
content = "####################### Start ########################"
for i in range(len(pages)):
    imagetext = pt.image_to_string(pages[i], lang='eng')##, config=tessdata_dir_config)
    content = content + "################### page " + str(i) + "####################" + chr(13)
    content = content + imagetext

In [31]:
#content_df = pd.DataFrame(data=content)

In [32]:
x = content.split("\n")

In [33]:
content_df = pd.DataFrame(data=x,columns=['text'])

In [34]:
#content_df.fillna("-----------------------",inplace=True)

In [35]:
content_df['row_id'] = content_df.index

In [36]:
content_df['length'] = content_df['text'].str.len()

In [37]:
content_df

,text,row_id,length
0,####################### Start ################...,0,124
1,,1,0
2,,2,1
3,,3,0
4,,4,1
...,...,...,...
1575,,1575,0
1576,Mastic Sealant,1576,14
1577,,1577,0
1578,Mastic sealant to tile abutment surfaces 1 m 4...,1578,59


In [38]:
#content_df['new_text'] = content_df['length'] > 0 content_df['text'] "-----------------------"

In [39]:
##df['hasimage'] = np.where(df['photos']!= '[]', True, False)

content_df['new_text'] = np.where(content_df['length'] > 0, content_df['text'], "-----------------------")

In [40]:
content_df.head(5)

,text,row_id,length,new_text
0,####################### Start ################...,0,124,####################### Start ################...
1,,1,0,-----------------------
2,,2,1,
3,,3,0,-----------------------
4,,4,1,


In [41]:
content_df[content_df['new_text'].str.match('\d\d\D\D\D\d\d.*')== True]

,text,row_id,length,new_text


In [42]:
content_df['date'] = np.where(content_df['new_text'].str.match('\d\d\D\D\D\d\d.*'),
                              content_df['text'].str[:7],
                              "-----------------------")
content_df['description'] = np.where(content_df['new_text'].str.match('\d\d\D\D\D\d\d.*'),
                              content_df['text'].str[8:],
                              "-----------------------")
content_df['useful'] = np.where(content_df['new_text'].str.match('\d\d\D\D\D\d\d.*'),
                              1,
                              0)

In [43]:
content_df[content_df['useful']==1]

,text,row_id,length,new_text,date,description,useful


In [21]:
numeric_const_pattern_1 = '[-+]? (?: (?: \d* \. \d+ ) | (?: \d+ \.? ) )(?: [Ee] [+-]? \d+ ) ?'
numeric_const_pattern = "(\d+\.\d+)"
content_df['amount_str'] = content_df['new_text'].str.extract(numeric_const_pattern)

In [22]:
#content_df[content_df['amount']>0]

content_df['amount'] = pd.to_numeric(content_df['amount_str'],errors='coerce')

content_df

,text,row_id,length,new_text,date,description,useful,amount_str,amount
0,####################### Start ################...,0,134,####################### Start ################...,-----------------------,-----------------------,0,NaN,NaN
1,,1,0,-----------------------,-----------------------,-----------------------,0,NaN,NaN
2,MR IM GOLDS,2,11,MR IM GOLDS,-----------------------,-----------------------,0,NaN,NaN
3,,3,0,-----------------------,-----------------------,-----------------------,0,NaN,NaN
4,33 WAKEFIELD GARDENS,4,20,33 WAKEFIELD GARDENS,-----------------------,-----------------------,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3005,,3005,0,-----------------------,-----------------------,-----------------------,0,NaN,NaN
3006,"5.00 2,388.47",3006,13,"5.00 2,388.47",-----------------------,-----------------------,0,5.00,5.0
3007,,3007,0,-----------------------,-----------------------,-----------------------,0,NaN,NaN
3008,17/11/2021,3008,10,17/11/2021,-----------------------,-----------------------,0,NaN,NaN
